In [1]:
import gc
import os
import pickle
import random
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import losses, optimizers, metrics, callbacks, Model, layers, backend as K
import SpeechModels
from augment_layers import FreqMaskLayer, TimeMaskLayer


In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
SEED = 123
N_CLASS = 12
MAX_EPOCHS = 200
TEST_PATH = "tensorflow-speech-recognition-challenge/test"
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
train_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/train",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/val",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True
)

test_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/test",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True
)

label_names = np.array(train_ds.class_names)
print("label names:", label_names)

Found 45586 files belonging to 12 classes.
Found 6513 files belonging to 12 classes.
Found 13024 files belonging to 12 classes.
label names: ['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']


In [5]:
test_dataset = tf.keras.utils.audio_dataset_from_directory(
    directory=TEST_PATH,
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

Found 158538 files belonging to 1 classes.


In [6]:
test_dataset.class_names

['audio']

In [7]:
train_dataset = train_ds.concatenate(val_ds)
train_dataset.element_spec

(TensorSpec(shape=(None, 16000, None), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [8]:
validation_dataset = test_ds
validation_dataset.element_spec

(TensorSpec(shape=(None, 16000, None), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [9]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels


train_dataset = train_dataset.map(squeeze, tf.data.AUTOTUNE)
validation_dataset = validation_dataset.map(squeeze, tf.data.AUTOTUNE)
test_dataset = test_dataset.map(squeeze, tf.data.AUTOTUNE)


def create_model(freq=False, time=False):
    m = SpeechModels.get_melspec_model(iLen=16000)
    m.trainable = False
    inputs, outputs = m.inputs, m.outputs

    x = m(inputs)
    if freq:
        x = FreqMaskLayer(10)(x)
    if time:
        x = TimeMaskLayer(10)(x)
    x = tf.expand_dims(x, axis=-1, name='mel_stft')

    x = layers.Conv2D(10, (5, 1), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)

    # x = Reshape((125, 80)) (x)
    # keras.backend.squeeze(x, axis)
    x = layers.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True)
                             )(x)  # [b_s, seq_len, vec_dim]
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True)
                             )(x)  # [b_s, seq_len, vec_dim]

    x_first = layers.Lambda(lambda q: q[:, -1])(x)  # [b_s, vec_dim]
    query = layers.Dense(128)(x_first)

    # dot product attention
    att_scores = layers.Dot(axes=[1, 2])([query, x])
    att_scores = layers.Softmax(name='attSoftmax')(att_scores)  # [b_s, seq_len]

    # rescale sequence
    att_vector = layers.Dot(axes=[1, 1])([att_scores, x])  # [b_s, vec_dim]
    x = layers.Dropout(rate=0.3)(att_vector)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(rate=0.3)(x)
    x = layers.Dense(32)(x)
    x = layers.Dropout(rate=0.3)(x)
    output = layers.Dense(N_CLASS, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model


model = create_model(freq=True)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 normalized_spectrogram_model (  (None, 125, 80)     0           ['input[0][0]']                  
 Functional)                                                                                      
                                                                                                  
 freq_mask_layer (FreqMaskLayer  (None, 125, 80)     0           ['normalized_spectrogram_model[0]
 )                                                               [0]']                            
                                                                                              

In [10]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)
model_checkpoint = callbacks.ModelCheckpoint(
    filepath="best_model_checkpoint",
    monitor='val_sparse_categorical_accuracy',
    mode='max',
    save_best_only=True)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=3,
                                        min_lr=0.00001, verbose=1)

In [11]:
history = model.fit(
    train_dataset,
    epochs=MAX_EPOCHS,
    validation_data=validation_dataset,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

Epoch 1/200
103/103 [==============================] - ETA: 0s - loss: 1.6506 - sparse_categorical_accuracy: 0.6201 - sparse_categorical_crossentropy: 1.6506

INFO:tensorflow:Assets written to: best_model_checkpoint\assets


INFO:tensorflow:Assets written to: best_model_checkpoint\assets


103/103 [==============================] - 91s 729ms/step - loss: 1.6506 - sparse_categorical_accuracy: 0.6201 - sparse_categorical_crossentropy: 1.6506 - val_loss: 1.6143 - val_sparse_categorical_accuracy: 0.6303 - val_sparse_categorical_crossentropy: 1.6143 - lr: 0.0010
Epoch 2/200
103/103 [==============================] - 50s 475ms/step - loss: 1.3160 - sparse_categorical_accuracy: 0.6439 - sparse_categorical_crossentropy: 1.3160 - val_loss: 1.8769 - val_sparse_categorical_accuracy: 0.4264 - val_sparse_categorical_crossentropy: 1.8769 - lr: 0.0010
Epoch 3/200
103/103 [==============================] - 48s 454ms/step - loss: 0.7574 - sparse_categorical_accuracy: 0.7548 - sparse_categorical_crossentropy: 0.7574 - val_loss: 2.0539 - val_sparse_categorical_accuracy: 0.3041 - val_sparse_categorical_crossentropy: 2.0539 - lr: 0.0010
Epoch 4/200
103/103 [==============================] - ETA: 0s - loss: 0.4539 - sparse_categorical_accuracy: 0.8572 - sparse_categorical_crossentropy: 0.4539

INFO:tensorflow:Assets written to: best_model_checkpoint\assets


...

INFO:tensorflow:Assets written to: best_model_checkpoint\assets


103/103 [==============================] - 70s 661ms/step - loss: 0.0787 - sparse_categorical_accuracy: 0.9795 - sparse_categorical_crossentropy: 0.0787 - val_loss: 0.1342 - val_sparse_categorical_accuracy: 0.9682 - val_sparse_categorical_crossentropy: 0.1342 - lr: 1.2500e-04
Epoch 29/200
103/103 [==============================] - ETA: 0s - loss: 0.0782 - sparse_categorical_accuracy: 0.9788 - sparse_categorical_crossentropy: 0.0782

INFO:tensorflow:Assets written to: best_model_checkpoint\assets


INFO:tensorflow:Assets written to: best_model_checkpoint\assets


103/103 [==============================] - 70s 668ms/step - loss: 0.0782 - sparse_categorical_accuracy: 0.9788 - sparse_categorical_crossentropy: 0.0782 - val_loss: 0.1318 - val_sparse_categorical_accuracy: 0.9686 - val_sparse_categorical_crossentropy: 0.1318 - lr: 1.2500e-04
Epoch 30/200
103/103 [==============================] - 49s 464ms/step - loss: 0.0741 - sparse_categorical_accuracy: 0.9805 - sparse_categorical_crossentropy: 0.0741 - val_loss: 0.1410 - val_sparse_categorical_accuracy: 0.9674 - val_sparse_categorical_crossentropy: 0.1410 - lr: 1.2500e-04
Epoch 31/200
103/103 [==============================] - 49s 465ms/step - loss: 0.0749 - sparse_categorical_accuracy: 0.9796 - sparse_categorical_crossentropy: 0.0749 - val_loss: 0.1372 - val_sparse_categorical_accuracy: 0.9667 - val_sparse_categorical_crossentropy: 0.1372 - lr: 1.2500e-04
Epoch 32/200
103/103 [==============================] - ETA: 0s - loss: 0.0764 - sparse_categorical_accuracy: 0.9795 - sparse_categorical_cross

In [12]:
predictions = model.predict(test_dataset)

310/310 [==============================] - 55s 174ms/step


In [13]:
predict_class = np.array(list(label_names[i] for i in np.argmax(predictions, axis=1)))

In [14]:
for i, (root, dir, fname) in enumerate(os.walk(os.path.join(TEST_PATH, 'audio'))):
    files = np.array(fname)

In [18]:
submission = pd.DataFrame(np.vstack([files, predict_class]).T, columns=["fname", "label"])
submission.to_csv("submission.csv", index=False)